In [26]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.insert(0, "../src/modeling")

import sklearn.tree
from matplotlib import pyplot as plt
%matplotlib inline

import model
import metrics
import train_test
import evaluation

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = pd.read_csv("../data/LendingClub/processed/Processed.csv", index_col=0)
data.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
id,,,,,,,,,,,,,,,,,,,,,
159170615,NaN,16000.0,16000.0,16000.0,36 months,17.74%,576.36,C,C5,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
159210254,NaN,10000.0,10000.0,10000.0,60 months,15.24%,239.17,C,C2,banquet server,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
159226028,NaN,15000.0,15000.0,15000.0,60 months,14.30%,351.37,C,C1,Chief Advisor Litigation Technology Unit,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
159328733,NaN,24000.0,24000.0,24000.0,36 months,10.33%,778.14,B,B1,Accountant,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
159374118,NaN,15000.0,15000.0,15000.0,36 months,11.02%,491.23,B,B2,Teacher,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["loan_status"]

id
159170615                                              Current
159210254                                           Fully Paid
159226028                                              Current
159328733                                              Current
159374118                                              Current
                                                       ...    
152956054                                              Current
153350995                                              Current
153062400                                              Current
Total amount funded in policy code 1: 2342814950           NaN
Total amount funded in policy code 2: 1003524233           NaN
Name: loan_status, Length: 143021, dtype: object

In [4]:
types = data.dtypes
numerical = types[(types == float) | (types==int)]
numerical

member_id                         float64
loan_amnt                         float64
funded_amnt                       float64
funded_amnt_inv                   float64
installment                       float64
                                   ...   
hardship_payoff_balance_amount    float64
hardship_last_payment_amount      float64
settlement_amount                 float64
settlement_percentage             float64
settlement_term                   float64
Length: 119, dtype: object

In [5]:
df = train_test.feature_selection(data, numerical.index)
print(df.shape)
df.head()

(143021, 119)


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,desc,dti,delinq_2yrs,fico_range_low,...,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
id,,,,,,,,,,,,,,,,,,,,,
159170615,NaN,16000.0,16000.0,16000.0,576.36,0.0,NaN,NaN,0.0,735.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159210254,NaN,10000.0,10000.0,10000.0,239.17,30000.0,NaN,36.88,0.0,775.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159226028,NaN,15000.0,15000.0,15000.0,351.37,115000.0,NaN,30.88,1.0,680.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159328733,NaN,24000.0,24000.0,24000.0,778.14,71000.0,NaN,11.41,1.0,670.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159374118,NaN,15000.0,15000.0,15000.0,491.23,65000.0,NaN,10.40,0.0,720.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for col in df.columns:
    df[col].fillna(df[col].median(), inplace=True)
    
df["loan_status"] = data["loan_status"]
df["loan_status"].fillna("Current", inplace=True)

In [18]:
df = df.dropna(axis=1)
df.isnull().any().sum()

0

In [19]:
X_train, X_valid, X_test, y_train, y_valid, y_test = train_test.prepare_data(df)

In [38]:
#svm = model.train_and_evaluate("svm", X_train, y_train, X_valid, y_valid)
dt, dt_accuracy = model.train_and_evaluate("decision_tree", X_train, y_train, X_valid, y_valid)
#rf = model.train_and_evaluate("random_forest", X_train, y_train, X_valid, y_valid)
#logreg = model.train_and_evaluate("log_reg", X_train, y_train, X_valid, y_valid)
#knn = model.train_and_evaluate("knn", X_train, y_train, X_valid, y_valid)
#naivebayes = model.train_and_evaluate("naive_bayes", X_train, y_train, X_valid, y_valid)

In [ ]:
model_performances = pd.DataFrame({
    "Model": ["SVC", "Decision Tree", "Random Forest", 
              "Logistic Regression", "K Nearest Neighbors", "Gaussian Naive Bayes"],
    "Accuracy": [svm, decisiontree, rf, logreg, knn, naivebayes]
})

model_performances.sort_values(by="Accuracy", ascending=False)

In [ ]:
BATCH_SIZE = 100
EPOCHS = 20

model.build_train_nn(X_train, y_train, X_valid, y_valid, EPOCHS, BATCH_SIZE)

In [36]:
fig = plt.figure(figsize=(25,20))
export_graphviz(dt, feature_names=df.columns[:-1])
plt.show()

<Figure size 1800x1440 with 0 Axes>

In [40]:
dt.coef_

AttributeError: 'DecisionTreeClassifier' object has no attribute 'coef_'

In [30]:
from sklearn.tree import export_graphviz